In [38]:
from PyBioMed.PyProtein import AAIndex, Autocorrelation
from tqdm import tqdm
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

# AAindex1

In [39]:
def AAindex1_descriptor(sequences: list):
    # initialize
    if not os.path.exists('aaindex1'):
        AAIndex.GetAAIndex1('ANDN920101')
    # get all AAindex name
    f = open('aaindex1')
    data = f.read()
    idxes = []
    for i, item in  enumerate(data.split('//')):
        if i == len(data.split('//')) -1:
            break
        else:
            idxes.append(item.split()[1])
    # create AAindex1 dictionary
    print('-'*100)
    print('crating all AAindex1 dictionary ...')
    AAindex1 = {}
    for idx in tqdm(idxes):
        AAindex1[idx] = AAIndex.GetAAIndex1(idx)
    print('completed')
    print('-'*100)
    # calculate AAindex1 descriptor
    print('-'*100)
    print('calculating AAindex1 descriptors ...')
    descriptor_matrix = np.zeros([len(sequences), len(AAindex1)])
    for i, seq in enumerate(tqdm(sequences, total=len(sequences))):
        for j, aaindex1_dict in enumerate(AAindex1.values()):
            ss = []
            for s in seq:
                try:
                    if aaindex1_dict[s] is not None:
                        ss.append(aaindex1_dict[s])
                except KeyError:
                    pass
            descriptor_matrix[i,j] = np.mean(ss)
    print('completed')
    print('-'*100)
    return pd.DataFrame(descriptor_matrix, columns = list(AAindex1.keys()))

In [86]:
f = open('aaindex1')
data = f.read()
idxes = []
for i, item in  enumerate(data.split('//')):
    if i == len(data.split('//')) -1:
        break
    else:
        idxes.append(item.split()[1])

In [87]:
len(idxes)

566

# AAindex23
ライブラリでファイルのパースがうまくいっていないようで, AAindex2,3については算出できない

# Autocorrelation

In [85]:
def acc_descriptor(sequences: list):
    descriptor_matrix = np.zeros([len(sequences), 720])
    for i, seq in enumerate(sequences):
        disc = Autocorrelation.CalculateAutoTotal(seq)
        descriptor_matrix[i] = np.array(list(disc.values()))
    return pd.DataFrame(descriptor_matrix, columns=disc.keys())

### 

# DeepLoc data set

In [5]:
df = pd.read_csv('../data/DeepLoc/DeepLocAll.csv')
sequences = df.iloc[:,0].tolist()
sequences = [''.join(s.split()) for s in sequences]
df.iloc[:,0] = sequences

,input,loc,membrane
0,MMKTLSSGNCTLNVPAKNSYRMVVLGASRVGKSSIVSRFLNGRFED...,Cell.membrane,M
1,MAKRTFSNLETFLIFLLVMMSAITVALLSLLFITSGTIENHKDLGG...,Cell.membrane,M
2,MGNCQAGHNLHLCLAHHPPLVCATLILLLLGLSGLGLGSFLLTHRT...,Cell.membrane,M
3,MDPSKQGTLNRVENSVYRTAFKLRSVQTLCQLDLMDSFLIQQVLWR...,Cell.membrane,M
4,MLLAWVQAFLVSNMLLAEAYGSGGCFWDNGHLYREDQTSPAPGLRC...,Cell.membrane,M
...,...,...,...
8459,IIECFFSCEIEKDGKSKEGKPCKPKGDKDKDKKCGGWRCKIKMCIKI,Extracellular,S
8460,MRVSVPVLALAFGSLAAAAPNAGRRTCGSVPPPEFFEASEKVAALE...,Extracellular,S
8461,MLFWTAFSMALSLRLALARSSIERGSTASDPQGDLLFLLDSSASVS...,Extracellular,S
8462,MMAFPPQSCVHVLPPKSIQMWEPNHNTMHGSSQSPRNYRVHDSQQM...,Extracellular,S
